---
# Medtronic(Soft Robotics)
# MCEN 4045, Fall 2021
# Tuesday November 16, 2021

The following command downloads the NDI Combined API C Interface Library from PyPi with out the need to complie the github repo found here: https://github.com/PlusToolkit/ndicapi

In [16]:
!pip install ndicapi

---
## Library import
We are referencing the code we found in the repo
secret sauce: https://github.com/PlusToolkit/ndicapi/blob/master/Applications/ndiBasicExample.py

In [17]:
import ndicapy # import is strange

In [18]:
from ndicapy import (
    ndiDeviceName, ndiProbe, NDI_OKAY,
    ndiOpen, ndiClose, ndiCommand, ndiGetError,
    ndiErrorString, NDI_115200,
    NDI_8N1, NDI_NOHANDSHAKE,
)

In [19]:
from IPython.display import clear_output

--- 
## Port Setup

In [45]:
# Number ports to check
MAX_SERIAL_PORTS = 20

name = ''
for port_no in range(MAX_SERIAL_PORTS):
    name = ndiDeviceName(port_no)
    
    if not name:
        continue
        
    result = ndiProbe(name) #Returns 257 if port not found, otherwise returns 1
    
    if result == NDI_OKAY: #NDI_OKAY == 0
        print(port_no)
        break
        
if result != NDI_OKAY:
    raise IOError(
        'Could not find any NDI device in '
        '{} serial port candidates checked. '
        'Please check the following:\n'
        '\t1) Is an NDI device connected to your computer?\n'
        '\t2) Is the NDI device switched on?\n'
        '\t3) Do you have sufficient privilege to connect to '
        'the device? (e.g. on Linux are you part of the "dialout" '
        'group?)'.format(MAX_SERIAL_PORTS)
    )

5


--- 
## Device Setup

In [46]:
device = ndiOpen(name) #seems like this command starts communication for API 
if not device:
    raise IOError(
        'Could not connect to NDI device found on '
        '{}'.format(name)
    )
print(device)

<polaris object 000002163161AFB0, COM6>


--- 
## Command Send Check

In [47]:
#Ensures the system configuuration was determine successfully
reply = ndiCommand(device, 'INIT:') 
error = ndiGetError(device)
if reply.startswith('ERROR') or error != NDI_OKAY:
    raise IOError(
        'Error when sending command: '
        '{}'.format(ndiErrorString(error))
    )
print(reply)

OKAY


In [23]:
# Sets serial communication settings
reply = ndiCommand(
    device,
    'COMM:{:d}{:03d}{:d}'.format(NDI_115200, NDI_8N1, NDI_NOHANDSHAKE)
)
print(reply)

OKAY


--- 
## Insert Commands HERE

Sensor part #:610059 T6F0-L00054

Puck Part# : 610066 T6D0-S00914
$\therefore$

In [48]:
def parser(reply):
    if "MISSING" in reply:
        print("MISSING")
        return 
    
    numHandles = reply[0:2]; 
    print("numHandles:" + numHandles);

    # <== Handle 1 ==>
    handle1 = reply[2:4]; 
    print("\nHandle1: " + handle1);

    # Quaternion
    Q0 = float(reply[5] + "." + reply[6:10]); 
    if (reply[4] == "-"):
        Q0 *= -1;

    Qx = float(reply[11] + "." + reply[12:16]); 
    if (reply[10] == "-"):
        Qx *= -1;

    Qy = float(reply[17] + "." + reply[18:22]); 
    if (reply[16] == "-"):
        Qy *= -1;        

    Qz = float(reply[23] + "." + reply[24:28]);
    if (reply[22] == "-"):
        Qz *= -1;
    print("\tQ0: " + str(Q0) + "\tQx: " + str(Qx) + "\tQy:" + str(Qy) + "\tQz:" + str(Qz));


    # Tool positioning
    Tx = float(reply[29:33] + "." + reply[33:35]); 
    if (reply[28] == "-"):
        Tx *= -1;

    Ty = float(reply[36:40] + "." + reply[40:42]); 
    if (reply[35] == "-"):
        Ty *= -1;

    Tz = float(reply[43:47] + "." + reply[47:49]); 
    if (reply[42] == "-"):
        Tz *= -1;

    print("\tTx: " + str(Tx) + "\tTy: " + str(Ty) + "\tTz:" + str(Tz));

    # Indicator value
    indValue = float(reply[50] + "." + reply[50:55]);
    if (reply[49] == "-"):
        Tz *= -1;
    print("\tIndicator value: " + str(indValue));



    # <== Handle 2 ==>
    handle2 = reply[72:74]; 
    print("\nHandle2: " + handle2);

    # Quaternion
    Q0 = float(reply[75] + "." + reply[76:80]); 
    if (reply[74] == "-"):
        Q0 *= -1;

    Qx = float(reply[81] + "." + reply[82:86]); 
    if (reply[80] == "-"):
        Qx *= -1;

    Qy = float(reply[87] + "." + reply[88:92]); 
    if (reply[86] == "-"):
        Qy *= -1;        

    Qz = float(reply[93] + "." + reply[94:98]);
    if (reply[92] == "-"):
        Qz *= -1;
    print("\tQ0: " + str(Q0) + "\tQx: " + str(Qx) + "\tQy:" + str(Qy) + "\tQz:" + str(Qz));

    # Tool positioning
    Tx = float(reply[99:103] + "." + reply[103:105]); 
    if (reply[98] == "-"):
        Tx *= -1;

    Ty = float(reply[106:110] + "." + reply[110:112]); 
    if (reply[105] == "-"):
        Ty *= -1;

    Tz = float(reply[113:117] + "." + reply[117:119]); 
    if (reply[112] == "-"):
        Tz *= -1;

    print("\tTx: " + str(Tx) + "\tTy: " + str(Ty) + "\tTz:" + str(Tz));

    # Indicator value
    indValue = float(reply[120] + "." + reply[121:126]);
    if (reply[119] == "-"):
        Tz *= -1;
    print("\tIndicator value: " + str(indValue));

In [49]:
num_setup_complete = 0
while(num_setup_complete < 2):
    To_Free = ndiCommand(device, 'PHSR:01')

    if(To_Free == '00'):
        To_Initialized = ndiCommand(device, 'PHSR:02')

        if(To_Initialized == '00'):
            To_enable = ndiCommand(device, 'PHSR:03')
            print(To_enable[2:4])
            if(To_enable == '00'):
                break
            else:
                if(To_enable[2:4] == '0B'):
                    ndiCommand(device, 'PENA:{}{}'.format(To_enable[2:4], 'S'))
                else:
                    ndiCommand(device, 'PENA:{}{}'.format(To_enable[2:4], 'D'))
                
                print("Success:")
                print(ndiCommand(device, 'PHINF:{}{}'.format(To_enable[2:4], '0001')))
                num_setup_complete += 1
        else:
            #There needs to be ports to be initialized
            ndiCommand(device, 'PINIT:{}'.format(To_Initialized[2:4]))
    else:
        #ports need to be freed
        print("free ports please")

0A
Success:
0B000000NDI-MF2     0003A21F0003F
0B
Success:
01000000NDI-MF2     0003A910C003F


In [51]:
reply4 = ndiCommand(device, 'TSTART:')
print(reply4)

OKAY


In [42]:
while(True):
    reply5 = ndiCommand(device, 'TX:')
    clear_output(wait=True)
    print(reply5)
    parser(reply5)
print(len(reply5))

020A+00894-02001-08646-04519+013955+002094-019293+006470002003F000D88DE
0B+00271-05979+07972+00776-003560-016639-010272+027560002003D000D88DE
0000
numHandles:02

Handle1: 0A
	Q0: 0.0894	Qx: -0.2001	Qy:-0.8646	Qz:-0.4519
	Tx: 139.55	Ty: 20.94	Tz:-192.93
	Indicator value: 0.00647

Handle2: 0B
	Q0: 0.0271	Qx: -0.5979	Qy:0.7972	Qz:0.0776
	Tx: -35.6	Ty: -166.39	Tz:-102.72
	Indicator value: 0.2756


KeyboardInterrupt: 

In [52]:
reply = ndiCommand(device, 'TSTOP:') 
print(reply)

OKAY


--- 
## Terminate Communication

In [53]:
ndiClose(device)

--- 
## API commands
Secret sauce #2: https://github.com/lara-unb/APIs-NDI-Digital/blob/master/Aurora_API_Guide.pdf

In [ ]:
pip install IPython